## Demo Notebook: How to calculate your own randomized PSD-based WFE
This notebook will show you how to calculate the full uncropped PSD WFE pre-calculated for the n0s0 randomizer seed set and saved somewhere you can access them.

**CAUTION**: This notebook will consume a lot of memory (5-6 GB), so don't run this notebook unless necessary. The saved uncropped PSD WFEs take up less than 1 GB of space.

**CONTEXT**: The work I did for MagAO-X in the Fresnel model paper uses an older, unpublished version of the PowerSpectrumWFE code in the POPPY github repo. The only major change is how the surface rms is scaled. It's minor, but there's a nonzero possibility the official version will slightly scale the OPD WFE different enough to not produce the same results. The WFEs are pretty large files, so it's a bit easier for you to generate your own instead of downloading them.

**BEFORE YOU START**: Before you run this notebook, you need to go into your poppy code and in the `poppy/wfe.py` file, copy+paste the `PowerSpectrumWFE_old` class from the `data/PSD_WFE/PowerSpectrumWFE_old.py` file, then save. This will let you calculate the OPDs based on the original PSD WFE code.

In [1]:
import numpy as np
from astropy import units as u
from astropy.io import fits
import copy

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import matplotlib

# need to look up which version
import pickle

# the only thing you need from my code
from model_kit import magaoxFunctions as mf

Run a version of POPPY no newer than 0.9.2 or after March 1, 2021. The newer version of POPPY has a sign change for OPD to match with ZEMAX. To match the modeling, use a version older than that.

In [2]:
import poppy
poppy.__version__

'0.9.2.dev11+g1887dd5'

Load up all the parameters and file locations

In [3]:
# declare MagAO-X variables
fr_parm = {'wavelength': 656e-9 * u.m,
           'npix': 538, # sample size
           'beam_ratio': 0.25, # oversample
           'leak_mult': 0.01, # vAPP leakage multiplier
           'bump': True, # T/F to use the MagAO-X pupil with tweeter bump masked
           'wfe_data': 'allopd'} # other options: common

# set up prescription details, this is important for labeling later.
wavelen_str = str(np.round(fr_parm['wavelength'].to(u.nm).value).astype(int))
br = int(1/fr_parm['beam_ratio'])
parm_name = '{0:3}_{1:1}x_{2}nm'.format(fr_parm['npix'], br, wavelen_str)

# set up file locations
home_dir = '/home/jhen/XWCL/code/MagAOX/' # change for your MagAO-X Fresnel directory
data_dir = home_dir + 'data/'
rx_dir = data_dir + 'rxCSV/'

Calculate all the wavefronts at each optic using the aberration-free rxCSV file. 

WARNING: This will consume 5-6 GB of your memory, check that you have enough!

In [4]:
%%time
# load the aberration-free prescription file to generate wavefronts for WFE calculation.
dl_loc = rx_dir+'rx_magaox_NCPDM_sci_{0}_noap_nowfe.csv'.format(parm_name)
dl_sys = mf.makeRxCSV(dl_loc, print_names=False)

# This builds the entire FresnelOpticalSystem
magaox_dl = mf.csvFresnel(rx_csv = dl_sys,
                       samp = fr_parm['npix'],
                       oversamp = fr_parm['beam_ratio'],
                       home_folder = home_dir, # this is critical for navigating folders
                       break_plane = 'F69Sci', # you can chnage this to any other plane
                       bump = fr_parm['bump']) # properly builds the pupil with bump mask

# This calculates all the wavefronts and holds it in memory.
psf_dl, wf_dl = magaox_dl.calc_psf(wavelength=fr_parm['wavelength'].value, 
                            return_intermediates=True)

CPU times: user 1min 4s, sys: 13.4 s, total: 1min 17s
Wall time: 30.9 s


Preset and load all the necessary variables and data for calculating the PSD-based WFE.

In [5]:
# Presets
apply_reflection = True

# Set the output directory for the OPD WFE.
n_set = 0
s_set = 0
psd_wfe_dir = data_dir + 'PSD_WFE/n{0}/s{0}/'.format(n_set, s_set) # change to how you want.

# Initialize PSD parameters
import pickle # this needs to get changed to all data being in FITS format
psd_dir = home_dir + 'PSD/model_parameters/'
psd_label = ['pdr', 'm2m3', 'oap', 'fm1in', 'fm2in', 
             'fm19mm_glue', 'fm1in_glue', 'fm0p5in_glue']
psd_dict = {}
for n in range(0, len(psd_label)):
    psd_filename = psd_dir + 'psd_parms_{0}.pickle'.format(psd_label[n])
    with open(psd_filename, 'rb') as psd_data_file:
        psd_data = pickle.load(psd_data_file)
    psd_dict.update(psd_data)
    
# Insert seed randomizer values
seed_set = np.load(psd_dir+'seed_set_10_n{0}.npy'.format(n_set))

# choose the optics which require a OPD WFE. These are the Optical Element Numbers in the rxCSV file.
j_set = [2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 15, 17, 18, 19, 21, 25, 26, 27, 29, 31, 32, 33]

# load the rxCSV file that indicates what PSD models to use.
rx_opd_loc = rx_dir+'rx_magaox_NCPDM_sci_{0}_noap_{1}.csv'.format(parm_name, fr_parm['wfe_data']) # specific rx file
rx_sys_opd = mf.makeRxCSV(rx_opd_loc, print_names=False)

This next part will only work if you've copy+pasted the`PowerSpectrumWFE_old` class from the `data/PSD_WFE/PowerSpectrumWFE_old.py` file into POPPY's `poppy/wfe.py` file. This will let you calculate the OPDs based on the original PSD WFE code.

In [7]:
# chug through it
for ind in range(0, len(j_set)):
    j = j_set[ind]
    
    seed_val = seed_set[s_set][j]
    
    rx_opt_data = rx_sys_opd[j]
    psd_type = rx_opt_data['surf_PSD_filename']
    psd_parameters = psd_dict[psd_type]
    psd_weight = psd_dict[psd_type+'_weight']
    
    n_wf = wf_dl[j+1] # the 0th wf_dl is the aperture required by POPPY and not in the rxCSV.
    
    wfe_filename = 'wfe_psd_j{0}'.format(j)
    
    mf.write_psdwfe(wavefront=n_wf, 
                    rx_opt_data = rx_opt_data, 
                    seed_val = seed_val,
                    psd_parameters = psd_parameters, 
                    psd_weight = psd_weight, 
                    apply_reflection = apply_reflection,
                    wfe_folder = psd_wfe_dir,
                    wfe_filename = wfe_filename)
    
    print('PSD WFE saved for {0} complete'.format(rx_opt_data['Name']))

PSD WFE saved for M-2 complete
PSD WFE saved for M-3 complete
PSD WFE saved for F-1 complete
PSD WFE saved for K-1 complete
PSD WFE saved for K-2 complete
PSD WFE saved for K-3 complete
PSD WFE saved for F-2 complete
PSD WFE saved for OAP-0 complete
PSD WFE saved for OAP-1 complete
PSD WFE saved for F-3 complete
PSD WFE saved for OAP-2 complete
PSD WFE saved for OAP-3 complete
PSD WFE saved for F-4 complete
PSD WFE saved for F-5 complete
PSD WFE saved for OAP-4 complete
PSD WFE saved for F-6 complete
PSD WFE saved for OAP-5-1 complete
PSD WFE saved for F-7 complete
PSD WFE saved for OAP-5-2 complete
PSD WFE saved for F-11 complete
PSD WFE saved for OAP-5-3 complete
PSD WFE saved for F-12 complete
